<a href="https://colab.research.google.com/github/Jihaad2021/Paractical_Keras/blob/main/Developer_Guides/02.The_Sequential_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers 

In [3]:
## When to use a sequential model 
# A: one input tensor and one output tensor 

# define sequential model with 3 layers 
model=keras.Sequential(
    [
        layers.Dense(2,activation='relu', name='layer1'),
        layers.Dense(3,activation='relu', name='layer2'),
        layers.Dense(4,name='layer3'),
    ]
)
# call model on test input 
x=tf.ones((3,3))
y=model(x)

In [4]:
## A Sequential model is not appropriate when:
# Your model has multiple inputs or multiple outputs
# Any of your layers has multiple inputs or multiple outputs
# You need to do layer sharing
# You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [5]:
model.layers

In [6]:
# Sequential model incrementally via the add() method:
model=keras.Sequential()
model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(4))


In [7]:
#  pop() method to remove layers
model.pop() 
print(len(model.layers))

2


In [8]:
# Sequential constructor accepts a name argument
model=keras.Sequential(name='my_sequential')
model.add(layers.Dense(2, activation='relu', name='layer1'))
model.add(layers.Dense(3, activation='relu', name='layer2'))
model.add(layers.Dense(4, name='layer3'))


In [9]:
## Specifying the input shape in advance

# Generally, 
# all layers in Keras need to know the shape of their inputs 
# in order to be able to create their weights. 
# So when you create a layer like this, initially, 
# it has no weights

layer=layers.Dense(3)
layer.weights

[]

In [10]:
# call layer on a test input 
x=tf.ones((1,4))
y=layer(x)
layer.weights

[<tf.Variable 'dense_3/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.01333249, -0.18083793, -0.65777755],
        [-0.23751307, -0.57233477, -0.16139114],
        [-0.18834239,  0.7079414 , -0.7393804 ],
        [ 0.3594334 , -0.90392035,  0.8984387 ]], dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [12]:
# When you instantiate a Sequential model without an input shape, 
#it isn't "built": it has no weights 

model=keras.Sequential(
    [
        layers.Dense(2,activation='relu', name='layer1'),
        layers.Dense(3,activation='relu', name='layer2'),
        layers.Dense(4,name='layer3'),
    ]
) # no weights at this stage 



In [14]:
# call layer on a test input 
x=tf.ones((1,4))
y=model(x)
print("number of weight after calling the model:", len(model.weights))

number of weight after calling the model: 6


In [15]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (1, 2)                    10        
                                                                 
 layer2 (Dense)              (1, 3)                    9         
                                                                 
 layer3 (Dense)              (1, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [17]:
model=keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation='relu'))

model.summary()
# Note that the Input object is not displayed 
# as part of model.layers, since it isn't a layer

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [19]:
# A simple alternative is to just pass an input_shape argument to your first layer:

model=keras.Sequential()
model.add(layers.Dense(2, activation='relu', input_shape=(4,)))
model.summary()

# Models built with a predefined input shape like this 
# always have weights (even before seeing any data) 
# and always have a defined output shape

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [22]:
## A common debugging workflow: add() + summary()

# When building a new Sequential architecture,
# it's useful to incrementally stack layers with add() 
# and frequently print model summaries.

model=keras.Sequential()
model.add(keras.Input(shape=(250,250,3)))
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu" ))
model.add(layers.MaxPooling2D(3))

model.summary()

model.add(layers.Conv2D(32, 3, activation="relu" ))
model.add(layers.Conv2D(32, 3, activation="relu" ))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu" ))
model.add(layers.Conv2D(32, 3, activation="relu" ))
model.add(layers.MaxPooling2D(2))

model.summary() 

model.add(layers.GlobalMaxPooling2D())

model.add(layers.Dense(10))


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 123, 123, 32)      2432      
                                                                 
 conv2d_13 (Conv2D)          (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 40, 40, 32)       0         
 2D)                                                             
                                                                 
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 123, 123, 32)      2432      
                          

In [23]:
## What to do once you have a model
# 1. train your model, evaluate it and run inference 
# 2. save your model to disk and restore it 
# 3. speed up model training by leveraging multiple gpu 

In [24]:
## Feature extraction with a Sequential model

# These attributes can be used to do neat things, 
# like quickly creating a model that extracts the outputs 
# of all intermediate layers in a Sequential model

initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)


In [25]:
# Here's a similar example that only extract features from one layer:

initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)